In [1]:
!pip install tld 

In [2]:
#importing all necessary dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
from tld import get_tld
import re
import pickle
from sklearn.preprocessing import LabelEncoder
from flask import Flask, redirect, url_for, request, render_template

In [3]:
def firstDirSize(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0
    
def tldSize(tld):
    try:
        return len(tld)
    except:
        return -1
    
def digitTotal(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letterTotal(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

def dirTotal(url):
    urldir = urlparse(url).path
    return urldir.count('/')

In [4]:
def ip_exits(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        return -1
    else:
        return 1
    
def url_shortening_exists(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1

In [6]:
app = Flask(__name__, template_folder='', static_url_path='/Desktop/Pooja/static/')

@app.route('/')
def hello_world():
    return render_template("home.html")


@app.route('/result', methods = ['GET', 'POST'])
def shortlist():
     if request.method == 'POST':
        url = request.form['websites']
        data = []
        data.append(len(url))        
        data.append(len(urlparse(url).netloc))
        data.append(len(urlparse(url).path))
        data.append(firstDirSize(url))
        data.append(tldSize(get_tld(url,fail_silently=True)))
        data.append(url.count('-'))
        data.append(url.count('@'))
        data.append(url.count('?'))
        data.append(url.count('%'))
        data.append(url.count('.'))
        data.append(url.count('='))
        data.append(url.count('http'))
        data.append(url.count('https'))
        data.append(url.count('www'))
        data.append(digitTotal(url))
        data.append(letterTotal(url))
        data.append(dirTotal(url))
        data.append(ip_exits(url))
        data.append(url_shortening_exists(url))
        data = [0 if di is None else di for di in data]
        data=np.array(data).reshape(1, -1)
        with open("bestModel.pkl","rb") as f:
            model = pickle.load(f)
            f.close()
        prediction = np.round(model.predict(data),0)
        if prediction == 1:
            prediction = 'Real'
        else:
            prediction = 'Fake'
        print(prediction)
        return render_template("shortlist.html", url=url, prediction=prediction)

    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/May/2024 23:48:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2024 23:48:35] "GET /Desktop/Pooja/static/cross.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 23:48:35] "GET /Desktop/Pooja/static/right.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 23:48:35] "GET /dataset.csv HTTP/1.1" 404 -
